In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score, roc_auc_score,
    confusion_matrix, classification_report
)

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from keras import Input, Model
from keras.layers import (
    Embedding, Bidirectional, LSTM, Dropout, BatchNormalization,
    Conv1D, MaxPooling1D, GlobalMaxPooling1D, Flatten, Dense, GRU
)
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import gc
#warnings
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
splits = {'train': 'cleaned_train_binary.csv', 'val': 'cleaned_val_binary.csv','test':'cleaned_test_binary.csv'}
train_df = pd.read_csv("hf://datasets/kcrl/Shared_Task_Fake_News_binary/" + splits["train"])
val_df = pd.read_csv("hf://datasets/kcrl/Shared_Task_Fake_News_binary/" + splits["val"])
test_df = pd.read_csv("hf://datasets/kcrl/Shared_Task_Fake_News_binary/" + splits["test"])

In [ ]:
with pd.option_context('display.max_colwidth',0):
    display(train_df.sample(n=10))

,text,label
2466,suprtthiruvadira,original
295,thableeq prasthanathanam katiya chetatharam yadartha muslingal utharavadi alla.avare arrest cheyd jailil adakanam.indian muslims ennum rajyathe janangalude koodeyanu.,Fake
2,അന്വേഷണം കഴിയുമ്പോൾ. c. a. a. യ്ക്ക് എതിരായ കലാപം പോലെ ആകുമോ. സ്വന്തം ആളെ തന്നെ പിടിച്ചു അകത്തിടേണ്ടി വരുമോ?,Fake
208,anubavicho,original
1233,വാക്സിൻ്റെ side effect ഒരു<br>ദിവസം കൊണ്ടല്ല അതു<br>വർഷങ്ങൾ കൊണ്ടാണ്<br>സംഭവിക്കുന്നത് !,Fake
1766,അവിടെ ഇനി മരിക്കാൻ ആരുമി ല്ലായിരിക്കാം,Fake
871,ആശാന് അടുപ്പിലും തൂറാമെങ്കിൽ.,original
1362,low class. low standard. govrt.vote cheyyan pottanmarum,original
1351,തീട്ടം തിന്നാൻ പൊയ് ക്കൂടെ,original
2787,konnu kalayum atra thanee .,Fake


In [ ]:
train_df.label.unique()

array(['Fake', 'original'], dtype=object)

In [ ]:
labeling = {
     'Fake':0,
    'original':1
}
train_df['label'] = train_df['label'].apply(lambda x : labeling[x])
test_df['label'] = test_df['label'].apply(lambda x : labeling[x])
val_df['label'] = val_df['label'].apply(lambda x : labeling[x])

In [ ]:
with pd.option_context('display.max_colwidth',0):
    display(train_df.sample(n=5))

,text,label
2028,superpower china,0
94,ഇവൾക്ക് ഒന്നും തിരുവാതിര എന്ന് പോലും അറിയില്ല അയ്യപ്പസ്വാമി യെ വരെ പണം ആയി കാണുന്ന വർഗം ആണ് സംസാകാരം എന്ത് എന്ന് പോലും ഇവർ കേട്ടിട്ടില്ല,1
2630,hitlor bharanam,1
828,എന്തിലെങ്കിലും ഉറച്ചു നിക്കടോ കമ്മ്യുണിസ്റ്റ് രാജ്യങ്ങളിൽ ഉണ്ടെന്നോ ഉണ്ടായിട്ട് മറച്ചു വെക്കുന്നതോ?,0
3086,china undakiya virus inu marun chinayuda kayillundu,0


In [ ]:
X_train = train_df['text']
y_train = train_df['label']
X_val = val_df['text']
y_val = val_df['label']
X_test = test_df['text']
y_test = test_df['label']

In [ ]:
# Convert the 'text' column to string type and handle missing values
X_train = train_df['text'].astype(str).fillna('')

# Tokenizing the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
vocab=tokenizer.word_index

In [ ]:
train_sequence = tokenizer.texts_to_sequences(X_train)
val_sequence = tokenizer.texts_to_sequences(X_val)
test_sequence = tokenizer.texts_to_sequences(X_test)

In [ ]:
maxlen = 120
train_data = pad_sequences(train_sequence, maxlen=maxlen,  padding='post')
val_data = pad_sequences(val_sequence, maxlen=maxlen,  padding='post')
test_data = pad_sequences(test_sequence, maxlen=maxlen,  padding='post')

In [ ]:
train_labels = y_train
val_labels = y_val
test_labels = y_test

In [ ]:
train_labels_reshaped = np.expand_dims(train_labels, axis=-1)
val_labels_reshaped = np.expand_dims(val_labels, axis=-1)
test_labels_reshaped = np.expand_dims(test_labels, axis=-1)

In [ ]:
vocab_size = len(tokenizer.word_index) + 1
input_shape = (maxlen,)

In [ ]:
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Dropout, BatchNormalization, Dense, Flatten
from keras.regularizers import l2

# Define the embedding layer
embedding_layer = Embedding(
    input_dim=vocab_size,
    output_dim=256,
    input_length=maxlen,
    trainable=True  # Allow fine-tuning of embeddings
)

# Define the input layer
email_input = Input(shape=(maxlen,), name='Email_input')

# Apply the embedding layer to the input
embedded_email = embedding_layer(email_input)

# Define the Bidirectional LSTM layers
lstm_layer1 = LSTM(units=64, return_sequences=True)
lstm_layer2 = LSTM(units=32, return_sequences=True)

# Apply the first LSTM layer
lstm_output1 = lstm_layer1(embedded_email)

# Apply dropout after the first LSTM layer
dropout1 = Dropout(0.25)(lstm_output1)

# Apply batch normalization
bn1 = BatchNormalization()(dropout1)

# Apply the second LSTM layer
lstm_output2 = lstm_layer2(bn1)

# Flatten the output
flatten_output = Flatten()(lstm_output2)

# Add a dense layer for multiclass classification
output = Dense(1, activation='softmax', kernel_regularizer=l2(0.01))(flatten_output)

# Create the model
lstm_model = Model(inputs=email_input, outputs=output)

# Compile the model
# Compile the model with the correct loss function
lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Display the model summary
lstm_model.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ Email_input (InputLayer)             │ (None, 120)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_2 (Embedding)              │ (None, 120, 256)            │       4,288,768 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_4 (LSTM)                        │ (None, 120, 64)             │          82,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 120, 64)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 120, 64)             │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_5 (LSTM)                        │ (None, 120, 32)             │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 3840)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │           3,841 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,387,457 (16.74 MB)

 Trainable params: 4,387,329 (16.74 MB)

 Non-trainable params: 128 (512.00 B)

In [ ]:
stopper= EarlyStopping(patience=5, monitor='val_accuracy')
checkpointer = ModelCheckpoint(filepath='model.weights.best.keras', verbose=3, save_best_only=True)

history=lstm_model.fit(
         train_data, train_labels_reshaped,
         batch_size=64,
         epochs=15,
         validation_data=(val_data, val_labels_reshaped),
         callbacks=[stopper, checkpointer],
         )

Epoch 1/15
49/51 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.5009 - loss: 0.6621
Epoch 1: val_loss improved from inf to 0.64725, saving model to model.weights.best.keras
51/51 ━━━━━━━━━━━━━━━━━━━━ 9s 30ms/step - accuracy: 0.5014 - loss: 0.6582 - val_accuracy: 0.5018 - val_loss: 0.6473
Epoch 2/15
48/51 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.5176 - loss: 0.1569
Epoch 2: val_loss improved from 0.64725 to 0.54921, saving model to model.weights.best.keras
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.5170 - loss: 0.1560 - val_accuracy: 0.5018 - val_loss: 0.5492
Epoch 3/15
48/51 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.4974 - loss: 0.0576
Epoch 3: val_loss improved from 0.54921 to 0.47751, saving model to model.weights.best.keras
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.4984 - loss: 0.0571 - val_accuracy: 0.5018 - val_loss: 0.4775
Epoch 4/15
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.4894 - loss: 0.0331
Epoch 4: val_loss did not improve fro

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Get predictions (class indices for each sample)
pred = lstm_model.predict(test_data)
y_pred = np.argmax(pred, axis=1)  # Get the class with the highest probability

# Calculate metrics
precision = precision_score(y_test, y_pred, average='micro')  # Micro-average precision
recall = recall_score(y_test, y_pred, average='micro')        # Micro-average recall
f1 = f1_score(y_test, y_pred, average='micro')                # Micro-average F1 score
accuracy = accuracy_score(y_test, y_pred)                     # Accuracy

# Print results
print("Precision (Micro):", precision)
print("Recall (Micro):", recall)
print("F1 Score (Micro):", f1)
print("Accuracy:", accuracy)


32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step
Precision (Micro): 0.4975466143277723
Recall (Micro): 0.4975466143277723
F1 Score (Micro): 0.4975466143277723
Accuracy: 0.4975466143277723


In [ ]:
embedding_layer = Embedding(
    input_dim=vocab_size,
    output_dim=300,
    input_length=maxlen,
    trainable=True  # Allow fine-tuning of embeddings
)
# Define the input layer
email_input = Input(shape=(maxlen,), name='Email_input')

# Apply the embedding layer to the input
embedded_email = embedding_layer(email_input)

# Define the Bidirectional LSTM layers
lstm_layer1 = Bidirectional(LSTM(units=256, return_sequences=True))
lstm_layer2 =Bidirectional(LSTM(units=128, return_sequences=True))

# Apply the first LSTM layer
lstm_output1 = lstm_layer1(embedded_email)

# Apply dropout after the first LSTM layer
dropout1 = Dropout(0.25)(lstm_output1)

# Apply batch normalization
bn1 = BatchNormalization()(dropout1)

# Apply the second LSTM layer
lstm_output2 = lstm_layer2(bn1)

flatten_output = Flatten()(lstm_output2)

# Add a dense layer for binary classification
output = Dense(7, activation='softmax', kernel_regularizer=l2(0.01))(flatten_output)

# Create the model
bilstm_model = Model(inputs=email_input, outputs=output)

# Compile the model
bilstm_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
bilstm_model.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ Email_input (InputLayer)             │ (None, 120)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_3 (Embedding)              │ (None, 120, 300)            │       5,025,900 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 120, 512)            │       1,140,736 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 120, 512)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 120, 512)            │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 120, 256)            │         656,384 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 30720)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 7)                   │         215,047 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,040,115 (26.86 MB)

 Trainable params: 7,039,091 (26.85 MB)

 Non-trainable params: 1,024 (4.00 KB)

In [ ]:
stopper= EarlyStopping(patience=5, monitor='val_accuracy')
checkpointer = ModelCheckpoint(filepath='model.weights.best.keras', verbose=3, save_best_only=True)

history=bilstm_model.fit(
         train_data, train_labels_reshaped,
         batch_size=64,
         epochs=15,
         validation_data=(val_data, val_labels_reshaped),
         callbacks=[stopper, checkpointer],
         )

Epoch 1/15
50/51 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.5058 - loss: 1.0965
Epoch 1: val_loss improved from inf to 1.00067, saving model to model.weights.best.keras
51/51 ━━━━━━━━━━━━━━━━━━━━ 7s 76ms/step - accuracy: 0.5085 - loss: 1.0884 - val_accuracy: 0.5436 - val_loss: 1.0007
Epoch 2/15
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.9213 - loss: 0.2917
Epoch 2: val_loss did not improve from 1.00067
51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 61ms/step - accuracy: 0.9215 - loss: 0.2910 - val_accuracy: 0.7926 - val_loss: 1.1875
Epoch 3/15
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.9813 - loss: 0.1017
Epoch 3: val_loss improved from 1.00067 to 0.88561, saving model to model.weights.best.keras
51/51 ━━━━━━━━━━━━━━━━━━━━ 5s 67ms/step - accuracy: 0.9814 - loss: 0.1015 - val_accuracy: 0.7988 - val_loss: 0.8856
Epoch 4/15
50/51 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.9958 - loss: 0.0530
Epoch 4: val_loss improved from 0.88561 to 0.87581, saving model to model.weights.b

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Get predictions (class indices for each sample)
pred = bilstm_model.predict(test_data)
y_pred = np.argmax(pred, axis=1)  # Get the class with the highest probability

# Calculate metrics
precision = precision_score(y_test, y_pred, average='micro')  # Micro-average precision
recall = recall_score(y_test, y_pred, average='micro')        # Micro-average recall
f1 = f1_score(y_test, y_pred, average='micro')                # Micro-average F1 score
accuracy = accuracy_score(y_test, y_pred)                     # Accuracy

# Print results
print("Precision (Micro):", precision)
print("Recall (Micro):", recall)
print("F1 Score (Micro):", f1)
print("Accuracy:", accuracy)


32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step
Precision (Micro): 0.7899901864573111
Recall (Micro): 0.7899901864573111
F1 Score (Micro): 0.7899901864573111
Accuracy: 0.7899901864573111


In [ ]:
embedding_layer = Embedding(
    input_dim=vocab_size,
    output_dim=300,
    input_length=maxlen,
    trainable=True  # Allow fine-tuning of embeddings
)
email_input = Input(shape=(maxlen,), name='Email_input')

# Apply the embedding layer to the input
embedded_email = embedding_layer(email_input)

# Define and apply the Bidirectional LSTM layer
lstm_layer = Bidirectional(LSTM(units=256, return_sequences=True))(embedded_email)

# Apply dropout after LSTM
dropout1 = Dropout(0.3)(lstm_layer)

# Apply batch normalization
bn1 = BatchNormalization()(dropout1)

# Add CNN layers after BiLSTM
# First CNN block
conv1 = Conv1D(filters=128, kernel_size=3, activation='relu', padding='same')(bn1)
pool1 = MaxPooling1D(pool_size=2)(conv1)

# Second CNN block
conv2 = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(pool1)
pool2 = MaxPooling1D(pool_size=2)(conv2)

# Apply global max pooling
global_pool = GlobalMaxPooling1D()(pool2)

# Add dropout for regularization
dropout2 = Dropout(0.2)(global_pool)

# Add batch normalization
bn2 = BatchNormalization()(dropout2)

# Add a dense layer for binary classification
output = Dense(7, activation='softmax', kernel_regularizer=l2(0.001))(bn2)

# Create the model
bilstm_cnn_model = Model(inputs=email_input, outputs=output)

# Compile the model
bilstm_cnn_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Display the model summary
bilstm_cnn_model.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ Email_input (InputLayer)             │ (None, 120)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_5 (Embedding)              │ (None, 120, 300)            │       5,125,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_4 (Bidirectional)      │ (None, 120, 256)            │         439,296 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 120, 256)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 120, 256)            │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d (Conv1D)                      │ (None, 120, 128)            │          98,432 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 60, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 60, 64)              │          24,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_1 (MaxPooling1D)       │ (None, 30, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d                 │ (None, 64)                  │               0 │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 7)                   │             455 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,689,603 (21.70 MB)

 Trainable params: 5,688,963 (21.70 MB)

 Non-trainable params: 640 (2.50 KB)

In [ ]:
stopper= EarlyStopping(patience=5, monitor='val_accuracy')
checkpointer = ModelCheckpoint(filepath='model.weights.best.keras', verbose=3, save_best_only=True)

history=bilstm_cnn_model.fit(
         train_data, train_labels_reshaped,
         batch_size=64,
         epochs=15,
         validation_data=(val_data, val_labels_reshaped),
         callbacks=[stopper, checkpointer],
         )

Epoch 1/15
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.1624 - loss: 2.5198
Epoch 1: val_loss improved from inf to 2.35855, saving model to model.weights.best.keras
84/84 ━━━━━━━━━━━━━━━━━━━━ 8s 40ms/step - accuracy: 0.1627 - loss: 2.5183 - val_accuracy: 0.2114 - val_loss: 2.3585
Epoch 2/15
83/84 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.3332 - loss: 1.8255
Epoch 2: val_loss did not improve from 2.35855
84/84 ━━━━━━━━━━━━━━━━━━━━ 3s 30ms/step - accuracy: 0.3349 - loss: 1.8213 - val_accuracy: 0.2151 - val_loss: 2.4976
Epoch 3/15
82/84 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.6962 - loss: 0.9143
Epoch 3: val_loss improved from 2.35855 to 2.14740, saving model to model.weights.best.keras
84/84 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step - accuracy: 0.6971 - loss: 0.9125 - val_accuracy: 0.2279 - val_loss: 2.1474
Epoch 4/15
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.8718 - loss: 0.4685
Epoch 4: val_loss improved from 2.14740 to 2.02731, saving model to model.weights.b

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Get predictions (class indices for each sample)
pred = bilstm_cnn_model.predict(test_data)
y_pred = np.argmax(pred, axis=1)  # Get the class with the highest probability

# Calculate metrics
precision = precision_score(y_test, y_pred, average='micro')  # Micro-average precision
recall = recall_score(y_test, y_pred, average='micro')        # Micro-average recall
f1 = f1_score(y_test, y_pred, average='micro')                # Micro-average F1 score
accuracy = accuracy_score(y_test, y_pred)                     # Accuracy

# Print results
print("Precision (Micro):", precision)
print("Recall (Micro):", recall)
print("F1 Score (Micro):", f1)
print("Accuracy:", accuracy)


17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Precision (Micro): 0.31066176470588236
Recall (Micro): 0.31066176470588236
F1 Score (Micro): 0.31066176470588236
Accuracy: 0.31066176470588236


In [ ]:
embedding_layer = Embedding(
    input_dim=vocab_size,
    output_dim=300,
    input_length=maxlen,
    trainable=True  # Allow fine-tuning of embeddings
)
email_input = Input(shape=(maxlen,), name='Email_input')

# Apply the embedding layer to the input
embedded_email = embedding_layer(email_input)

# Define and apply the Bidirectional LSTM layer
lstm_layer = Bidirectional(GRU(units=128, return_sequences=True))(embedded_email)

# Apply dropout after LSTM
dropout1 = Dropout(0.25)(lstm_layer)

# Apply batch normalization
bn1 = BatchNormalization()(dropout1)

# Add CNN layers after BiLSTM
# First CNN block
conv1 = Conv1D(filters=128, kernel_size=3, activation='relu', padding='same')(bn1)
pool1 = MaxPooling1D(pool_size=2)(conv1)

# Second CNN block
conv2 = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(pool1)
pool2 = MaxPooling1D(pool_size=2)(conv2)

# Apply global max pooling
global_pool = GlobalMaxPooling1D()(pool2)

# Add dropout for regularization
dropout2 = Dropout(0.2)(global_pool)

# Add batch normalization
bn2 = BatchNormalization()(dropout2)

# Add a dense layer for binary classification
output = Dense(7, activation='softmax', kernel_regularizer=l2(0.001))(bn2)

# Create the model
bigru_cnn_model = Model(inputs=email_input, outputs=output)

# Compile the model
bigru_cnn_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Display the model summary
bigru_cnn_model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ Email_input (InputLayer)             │ (None, 120)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_5 (Embedding)              │ (None, 120, 300)            │       5,025,900 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_3 (Bidirectional)      │ (None, 120, 256)            │         330,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 120, 256)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_6                │ (None, 120, 256)            │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_2 (Conv1D)                    │ (None, 120, 128)            │          98,432 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_2 (MaxPooling1D)       │ (None, 60, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_3 (Conv1D)                    │ (None, 60, 64)              │          24,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_3 (MaxPooling1D)       │ (None, 30, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d_1               │ (None, 64)                  │               0 │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_7                │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 7)                   │             455 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,480,947 (20.91 MB)

 Trainable params: 5,480,307 (20.91 MB)

 Non-trainable params: 640 (2.50 KB)

In [ ]:
stopper= EarlyStopping(patience=5, monitor='val_accuracy')
checkpointer = ModelCheckpoint(filepath='model.weights.best.keras', verbose=3, save_best_only=True)

history=bigru_cnn_model.fit(
         train_data, train_labels_reshaped,
         batch_size=64,
         epochs=15,
         validation_data=(val_data, val_labels_reshaped),
         callbacks=[stopper, checkpointer],
         )

Epoch 1/15
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.2304 - loss: 2.3501
Epoch 1: val_loss improved from inf to 2.01639, saving model to model.weights.best.keras
51/51 ━━━━━━━━━━━━━━━━━━━━ 7s 56ms/step - accuracy: 0.2322 - loss: 2.3446 - val_accuracy: 0.1656 - val_loss: 2.0164
Epoch 2/15
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.8334 - loss: 0.6808
Epoch 2: val_loss improved from 2.01639 to 1.49500, saving model to model.weights.best.keras
51/51 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - accuracy: 0.8343 - loss: 0.6781 - val_accuracy: 0.5067 - val_loss: 1.4950
Epoch 3/15
50/51 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.9874 - loss: 0.1307
Epoch 3: val_loss improved from 1.49500 to 1.23697, saving model to model.weights.best.keras
51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - accuracy: 0.9874 - loss: 0.1303 - val_accuracy: 0.5448 - val_loss: 1.2370
Epoch 4/15
49/51 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.9962 - loss: 0.0584
Epoch 4: val_loss improved from 1.236

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Get predictions (class indices for each sample)
pred = bigru_cnn_model.predict(test_data)
y_pred = np.argmax(pred, axis=1)  # Get the class with the highest probability

# Calculate metrics
precision = precision_score(y_test, y_pred, average='micro')  # Micro-average precision
recall = recall_score(y_test, y_pred, average='micro')        # Micro-average recall
f1 = f1_score(y_test, y_pred, average='micro')                # Micro-average F1 score
accuracy = accuracy_score(y_test, y_pred)                     # Accuracy

# Print results
print("Precision (Micro):", precision)
print("Recall (Micro):", recall)
print("F1 Score (Micro):", f1)
print("Accuracy:", accuracy)


32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step
Precision (Micro): 0.788027477919529
Recall (Micro): 0.788027477919529
F1 Score (Micro): 0.788027477919529
Accuracy: 0.788027477919529
